In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

from climatools.dataset import *
import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
                
#    if 'band' in ds.dims:
#        try:
#            ds = ds.squeeze('band')
#        except ValueError:
#            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DCLIRAD.wgt_cool.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DLBLNEW.crd_cool),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DLBLNEW.wgt_cool.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
#    if 'band' in ds.dims:
#        try:
#            ds = ds.squeeze('band').drop('band')
#        except ValueError:
#            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DLBLNEW.crd_flux, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DLBLNEW.wgt_flux.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DCLIRAD.wgt_flux.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DLBLNEW.crd_cool, DCLIRAD.wgt_cool.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DLBLNEW.crd_flux, DCLIRAD.wgt_flux.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DLBLNEW.wgt_flux.coords['g']:
        df = DLBLNEW.wgt_flux.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DLBLNEW.wgt_cool.coords['g']:
        df = DLBLNEW.wgt_cool.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DLBLNEW = LBLnewBestfitModelData.from_mongodoc(doc)
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DLBLNEW = LBLnewOverlapModelData.from_mongodoc(doc)

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DCLIRAD = CliradnewLWModelData.from_mongodoc(doc)

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[3],3a
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.37556e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0, 0]"
ng_refs,-,"[3, 2, 4]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.389489                    1  0.243080
0.000750        2  0.388176                    2  0.247000
0.001052        3  0.399332                    3  0.257603
0.001476        4  0.409658                    4  0.268757
0.002070        5  0.417947                    5  0.279403
0.002904        6  0.422956                    6  0.289427
0.004074        7  0.424201                    7  0.298182
0.005714        8  0.420278                    8  0.305292
0.008015        9  0.409925                    9  0.309654
0.011243       10  0.391847                   10  0.310382
0.015771       11  0.364466                   11  0.305646
0.022122       12  0.326236                   12  0.293086
0.031031       13  0.275434                   13  0.269960
0.043528       14  0.210205                   14  0.233169
0.061057       15  0.160402                   15  0.206083
0.085645       16  0.178620                   16  0.231299
0.120136       17  0.241901                   17  0.287355
0.168516       18  0.291570                   18  0.316914
0.236378       19  0.331875                   19  0.318971
0.331549       20  0.378121                   20  0.323522
0.465100       21  0.448609                   21  0.373582
0.652400       22  0.574815                   22  0.506078
0.915100       23  0.721690                   23  0.666466
1.283650       24  0.691988                   24  0.660808
1.800600       25  0.535288                   25  0.525919
2.525700       26  0.414367                   26  0.412346
3.542800       27  0.325918                   27  0.337433
4.969550       28  0.257415                   28  0.282771
6.970850       29  0.205556                   29  0.238358
9.778100       30  0.164672                   30  0.192285
13.715850      31  0.127783                   31  0.144528
19.239350      32  0.094104                   32  0.099500
26.987250      33  0.071226                   33  0.069191
37.855300      34  0.057832                   34  0.055442
53.100050      35  0.044322                   35  0.047754
73.887500      36  0.029910                   36  0.038206
97.662500      37  0.021253                   37  0.030094
121.437500     38  0.016832                   38  0.023773
145.212500     39  0.008260                   39  0.012404
168.987500     40 -0.001416                   40  0.000322
192.762500     41  0.000408                   41  0.000976
216.537500     42  0.013739                   42  0.014038
240.312500     43  0.025656                   43  0.025789
264.087500     44  0.035534                   44  0.034998
287.862500     45  0.044365                   45  0.042659
311.637500     46  0.052432                   46  0.049154
335.412500     47  0.059858                   47  0.055120
359.187500     48  0.066717                   48  0.060782
382.962500     49  0.073051                   49  0.066358
406.737500     50  0.078825                   50  0.071848
430.512500     51  0.083967                   51  0.077201
454.287500     52  0.088361                   52  0.082243
478.062500     53  0.091971                   53  0.086951
501.837500     54  0.094914                   54  0.091181
525.612500     55  0.097388                   55  0.095000
549.387500     56  0.099557                   56  0.098625
573.162500     57  0.101520                   57  0.102030
596.937500     58  0.103342                   58  0.105196
620.712500     59  0.105074                   59  0.108114
644.487500     60  0.106757                   60  0.110757
668.262500     61  0.108432                   61  0.113189
692.037500     62  0.110170                   62  0.115433
715.812500     63  0.112058                   63  0.117718
739.587500     64  0.114182                   64  0.120029
763.362500     65  0.116627                   65  0.122444
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -29.581934   0.000000 -29.581934 -29.702917  4.413195e-08   
0.000624    2     -29.581952   0.000046 -29.581905 -29.702933  3.455173e-05   
0.000876    3     -29.581958   0.000064 -29.581894 -29.702940  4.855046e-05   
0.001229    4     -29.581968   0.000091 -29.581877 -29.702949  6.854541e-05   
0.001723    5     -29.581981   0.000128 -29.581853 -29.702962  9.712777e-05   
0.002417    6     -29.581999   0.000180 -29.581819 -29.702980  1.379283e-04   
0.003391    7     -29.582025   0.000254 -29.581770 -29.703005  1.960940e-04   
0.004757    8     -29.582059   0.000358 -29.581701 -29.703039  2.788354e-04   
0.006672    9     -29.582108   0.000502 -29.581606 -29.703087  3.962008e-04   
0.009359    10    -29.582176   0.000700 -29.581476 -29.703154  5.619899e-04   
0.013128    11    -29.582270   0.000970 -29.581301 -29.703249  7.949539e-04   
0.018415    12    -29.582404   0.001332 -29.581072 -29.703382  1.119868e-03   
0.025830    13    -29.582595   0.001810 -29.580786 -29.703573  1.567934e-03   
0.036232    14    -29.582874   0.002428 -29.580446 -29.703849  2.176491e-03   
0.050823    15    -29.583295   0.003211 -29.580083 -29.704256  2.986866e-03   
0.071291    16    -29.583932   0.004238 -29.579694 -29.704857  4.087390e-03   
0.100000    17    -29.584849   0.005762 -29.579086 -29.705699  5.716571e-03   
0.140271    18    -29.586078   0.008145 -29.577933 -29.706805  8.192941e-03   
0.196760    19    -29.587672   0.011691 -29.575981 -29.708202  1.171052e-02   
0.275997    20    -29.589660   0.016793 -29.572867 -29.709888  1.639091e-02   
0.387100    21    -29.591969   0.024080 -29.567890 -29.711769  2.253077e-02   
0.543100    22    -29.594254   0.034654 -29.559600 -29.713509  3.117444e-02   
0.761700    23    -29.595106   0.050390 -29.544716 -29.713819  4.459102e-02   
1.068500    24    -29.591321   0.072834 -29.518487 -29.709907  6.490261e-02   
1.498800    25    -29.582270   0.099055 -29.483215 -29.701278  8.995943e-02   
2.102400    26    -29.570761   0.125820 -29.444942 -29.690175  1.164640e-01   
2.949000    27    -29.557826   0.154439 -29.403386 -29.677384  1.450302e-01   
4.136600    28    -29.543780   0.186244 -29.357537 -29.662972  1.780929e-01   
5.802500    29    -29.528986   0.222247 -29.306739 -29.647073  2.180005e-01   
8.139200    30    -29.513547   0.263706 -29.249842 -29.629449  2.663606e-01   
11.417000   31    -29.497520   0.311618 -29.185903 -29.610025  3.216043e-01   
16.014700   32    -29.482221   0.365913 -29.116308 -29.590091  3.803925e-01   
22.464000   33    -29.471270   0.426855 -29.044416 -29.573704  4.400284e-01   
31.510500   34    -29.467531   0.499443 -28.968088 -29.564203  5.046825e-01   
44.200100   35    -29.472291   0.591134 -28.881157 -29.562869  5.866965e-01   
62.000000   36    -29.493049   0.705345 -28.787704 -29.577850  7.023782e-01   
85.775000   37    -29.545127   0.841658 -28.703468 -29.626678  8.588186e-01   
109.550000  38    -29.617933   0.974322 -28.643612 -29.699801  1.016705e+00   
133.325000  39    -29.706814   1.110606 -28.596207 -29.791811  1.175675e+00   
157.100000  40    -29.818628   1.245682 -28.572945 -29.909242  1.328042e+00   
180.875000  41    -29.964343   1.387409 -28.576934 -30.063068  1.482775e+00   
204.650000  42    -30.139695   1.563911 -28.575785 -30.246421  1.668877e+00   
228.425000  43    -30.329752   1.792660 -28.537091 -30.444358  1.906353e+00   
252.200000  44    -30.529194   2.064357 -28.464836 -30.650818  2.185452e+00   
275.975000  45    -30.736253   2.371491 -28.364761 -30.863877  2.497086e+00   
299.750000  46    -30.949555   2.709740 -28.239815 -31.082050  2.835411e+00   
323.525000  47    -31.167843   3.075694 -28.092149 -31.304160  3.195970e+00   
347.300000  48    -31.389935   3.466366 -27.923569 -31.528600  3.575662e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')